In [1]:
def load_dataset(file_name):
    import pandas as pd
    import json
    
    df = []
    with open('/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-clickbait-spoiling/' + file_name) as f:
        for i in f:
            i = json.loads(i)
            tweet = i['postText']
            label = i['tags']
            
            assert len(tweet) == 1
            tweet = tweet[0]
            
            assert len(label) == 1
            label = label[0]
            
            if label not in ['phrase', 'phrases', 'passage', 'multi']:
                print(label)
                
            assert label in ['phrase', 'phrases', 'passage', 'multi']
            
            if label == 'multi':
                continue
            
            df += [{'text': tweet, 'labels': (label == 'phrase' or label == 'phrases')}]

    return pd.DataFrame(df)   
            
    
test_dataset = load_dataset('test.jsonl')
train_dataset = load_dataset('train.jsonl')
validation_dataset = load_dataset('validation.jsonl')

In [6]:
test_dataset

,text,labels
0,This simple household item saves lives,True
1,You'll Never Believe Who Robert Downey Jr.'s '...,True
2,THE NEWEST THEORY ON HIDDLESWIFT'S ROMANCE IS ...,False
3,"The little girl from ""Titanic"" is all grown up...",True
4,Why we never really get over that first love,False
...,...,...
821,Inside our three-month effort to attend Apple'...,False
822,Dad And Son Are Seconds From Assassination By ...,False
823,Kanye West Reveals His Favorite Song... Of All...,False
824,"""Pippi Longstocking"" star arrested",True


In [7]:
train_dataset

,text,labels
0,"Wes Welker Wanted Dinner With Tom Brady, But P...",False
1,NASA sets date for full recovery of ozone hole,True
2,This is what makes employees happy -- and it's...,True
3,The perfect way to cook rice so that it's perf...,True
4,What happens if your new AirPods get lost or s...,False
...,...,...
2636,If You See A Purple Butterfly Sticker At The H...,False
2637,Has Facebook's video explosion completely shak...,False
2638,Cop Is Eating At A Chili's When Teen Hands Him...,False
2639,You need to see this Twitter account that pred...,True


In [8]:
validation_dataset

,text,labels
0,Five Nights at Freddy’s Sequel Delayed for Wei...,False
1,Here’s how much you should be tipping your hai...,True
2,A man swallowed a microSD card and you won't b...,False
3,This popular soda could cure your hangovers sc...,True
4,The anytime snack you won't feel guilty about ...,True
...,...,...
652,"Dog Dies One Hour After Hiking With His Owner,...",False
653,This is what happens when you leave a hotel cl...,False
654,This Texas GOP elector announces that he won't...,True
655,WikiLeaks' Julian Assange Reported Dead,False


# Training

In [2]:
configurations = []

for learn_rate in [1e-5, 4e-5, 1e-4]:
    for warumup_ratio in [0.02, 0.06, 0.1]:
        for batch_size in [8, 16, 32]:
            configurations += [{
                "overwrite_output_dir": True,
                "num_train_epochs": 10,
                "fp16": False,
                "train_batch_size": batch_size,
                "gradient_accumulation_steps": 4,
                "evaluate_during_training": True,
                "max_seq_length": 64,
                "learning_rate": learn_rate,
                "early_stopping_consider_epochs": True,
                "early_stopping_delta": 0.01,
                "early_stopping_metric": "acc",
                "early_stopping_metric_minimize": False,
                "early_stopping_patience": 3,
                "evaluate_during_training_steps": 331,
                "output_dir": "outputs/",
                'warmup_ratio': warumup_ratio,
                # 'warmup_steps': 0, # usually empfohlen, warmup_ratio ist prozentuales äquivalent 
                #                    # <> überschreiben sich gegenseitig
                'save_steps': 2000,
                "manual_seed": 12345
            }]

In [7]:
def train_model(config, num):
    from simpletransformers.classification import ClassificationModel
    import sklearn
    
    config["output_dir"] = "outputs/bert_" + str(num)
    
    model = ClassificationModel("bert", "bert-large-cased", args = config)
    # model.train_model(train_dataset)
    model.train_model(train_dataset, eval_df=validation_dataset, acc=sklearn.metrics.accuracy_score)

In [9]:
for config, num in zip(configurations, range(len(configurations))):
    print('Running configuration number', num)
    
    train_model(config, num)

Running configuration number 0


Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at 

  0%|          | 0/2641 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/331 [00:00<?, ?it/s]

...

  0%|          | 0/657 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/83 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/83 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

In [42]:
from simpletransformers.classification import ClassificationModel
model = ClassificationModel("roberta", "roberta-large", args = args)
# model.train_model(train_dataset)
model.train_model(train_dataset, eval_df=validation_dataset, acc=sklearn.metrics.accuracy_score)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

  0%|          | 0/2641 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

...

Running Epoch 8 of 10:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

(820,
 defaultdict(list,
             {'global_step': [82,
               164,
               246,
               300,
               328,
               410,
               492,
               574,
               600,
               656,
               738,
               820],
              'train_loss': [0.4724636971950531,
               0.23830775916576385,
               0.4726167321205139,
               0.324043333530426,
               0.04376447573304176,
               0.0019246878800913692,
               0.011581802740693092,
               0.0007018963224254549,
               0.0034602258820086718,
               0.00038723601028323174,
               0.0016906267264857888,
               9.881961887003854e-05],
              'mcc': [0.4379899140133909,
               0.5700232642378193,
               0.5278943599035283,
               0.5303111080019447,
               0.5986388216056269,
               0.5522782622892372,
               0.5474424397630353,
           

In [38]:
model.eval_model(validation_dataset, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/657 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/83 [00:00<?, ?it/s]

({'mcc': 0.5189907816932701,
  'tp': 264,
  'tn': 235,
  'fp': 87,
  'fn': 71,
  'auroc': 0.8332344488736442,
  'auprc': 0.8547997162455769,
  'acc': 0.7595129375951294,
  'eval_loss': 1.5449147986592473},
 array([[-4.50758696,  4.8231678 ],
        [-4.62452698,  4.63890123],
        [ 4.56198597, -4.33566332],
        ...,
        [ 0.98726308, -0.87757879],
        [ 3.27670145, -3.22399449],
        [-4.55029869,  4.87138462]]),
 [['Five Nights at Freddy’s Sequel Delayed for Weird Reason',
   'Here’s how much you should be tipping your hairdresser',
   "A man swallowed a microSD card and you won't believe what happened next!",
   'This popular soda could cure your hangovers scientists say:',
   "The anytime snack you won't feel guilty about eating",
   'You won\'t believe this stunning "Harry Potter" revelation about Professor McGonagall',
   "J.J. Abrams has an answer on if there will be a post-credits scene in the new 'Star Wars'",
   'Kristin Cavallari (@KristinCav) opens up abo

In [37]:
model.eval_model(test_dataset, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/826 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/104 [00:00<?, ?it/s]

({'mcc': 0.5662373938122638,
  'tp': 338,
  'tn': 309,
  'fp': 94,
  'fn': 85,
  'auroc': 0.83468548533751,
  'auprc': 0.8406305749647641,
  'acc': 0.7832929782082324,
  'eval_loss': 1.4019140609235854},
 array([[-4.73007059,  4.75768614],
        [-4.66822481,  4.89313078],
        [ 4.12303448, -3.80747771],
        ...,
        [-4.3079958 ,  4.59369469],
        [-4.60104084,  4.89709044],
        [-4.45188284,  4.57753611]]),
 [])

# Model Selection with Validation data

In [ ]:
from glob import glob
from simpletransformers.classification import ClassificationModel
import sklearn
import pandas as pd
from tqdm import tqdm

df = []

for config, num in zip(configurations, range(len(configurations))):
    directory = "outputs/bert_" + str(num) + "/"
    
    for checkpoint in tqdm(glob(directory + "checkpoint-*")):
        # print(checkpoint)
        model = ClassificationModel("bert", checkpoint)
        
        valid_acc = model.eval_model(validation_dataset, acc=sklearn.metrics.accuracy_score)[0]['acc']
        test_acc = model.eval_model(test_dataset, acc=sklearn.metrics.accuracy_score)[0]['acc']
        
        df += [{"checkpoint": checkpoint, "valid_acc": valid_acc, "test_acc": test_acc,
               "config": config}]
        
df = pd.DataFrame(df)
df

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/83 [00:00<?, ?it/s]

  0%|          | 0/826 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/104 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/83 [00:00<?, ?it/s]

  0%|          | 0/826 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/104 [00:00<?, ?it/s]

 67%|██████▋   | 8/12 [07:31<03:45, 56.39s/it]

  0%|          | 0/657 [00:00<?, ?it/s]

In [ ]:
df.to_json('outputs/bert_eval_results.csv')

In [ ]:
print('finished!')

In [16]:
df[df['checkpoint'].str.contains('epoch')].sort_values('valid_acc', ascending=False)

,checkpoint,valid_acc,test_acc,config
143,outputs/bert_12/checkpoint-164-epoch-2,0.783866,0.757869,"{'overwrite_output_dir': True, 'num_train_epoc..."
274,outputs/bert_24/checkpoint-574-epoch-7,0.780822,0.774818,"{'overwrite_output_dir': True, 'num_train_epoc..."
265,outputs/bert_24/checkpoint-656-epoch-8,0.773212,0.778450,"{'overwrite_output_dir': True, 'num_train_epoc..."
259,outputs/bert_23/checkpoint-80-epoch-4,0.773212,0.746973,"{'overwrite_output_dir': True, 'num_train_epoc..."
108,outputs/bert_9/checkpoint-492-epoch-6,0.767123,0.766344,"{'overwrite_output_dir': True, 'num_train_epoc..."
...,...,...,...,...
234,outputs/bert_21/checkpoint-410-epoch-5,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."
207,outputs/bert_18/checkpoint-492-epoch-6,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."
201,outputs/bert_18/checkpoint-410-epoch-5,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."
240,outputs/bert_21/checkpoint-492-epoch-6,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."


In [17]:
df.sort_values('test_acc', ascending=False)

,checkpoint,valid_acc,test_acc,config
265,outputs/bert_24/checkpoint-656-epoch-8,0.773212,0.778450,"{'overwrite_output_dir': True, 'num_train_epoc..."
274,outputs/bert_24/checkpoint-574-epoch-7,0.780822,0.774818,"{'overwrite_output_dir': True, 'num_train_epoc..."
132,outputs/bert_12/checkpoint-246-epoch-3,0.765601,0.772397,"{'overwrite_output_dir': True, 'num_train_epoc..."
109,outputs/bert_9/checkpoint-574-epoch-7,0.750381,0.771186,"{'overwrite_output_dir': True, 'num_train_epoc..."
294,outputs/bert_26/checkpoint-180-epoch-9,0.750381,0.771186,"{'overwrite_output_dir': True, 'num_train_epoc..."
...,...,...,...,...
283,outputs/bert_25/checkpoint-164-epoch-4,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."
234,outputs/bert_21/checkpoint-410-epoch-5,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."
243,outputs/bert_22/checkpoint-123-epoch-3,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."
242,outputs/bert_21/checkpoint-164-epoch-2,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."


In [18]:
for _, i in [i for i in df[df['checkpoint'].str.contains('epoch')].sort_values('valid_acc', ascending=False).iterrows()][:5]:
    print(i['valid_acc'], i['config'])

0.7838660578386606 {'overwrite_output_dir': True, 'num_train_epochs': 10, 'fp16': False, 'train_batch_size': 8, 'gradient_accumulation_steps': 4, 'evaluate_during_training': True, 'max_seq_length': 64, 'learning_rate': 4e-05, 'early_stopping_consider_epochs': True, 'early_stopping_delta': 0.01, 'early_stopping_metric': 'acc', 'early_stopping_metric_minimize': False, 'early_stopping_patience': 3, 'evaluate_during_training_steps': 331, 'output_dir': 'outputs/bert_12', 'warmup_ratio': 0.06, 'save_steps': 2000, 'manual_seed': 12345}
0.7808219178082192 {'overwrite_output_dir': True, 'num_train_epochs': 10, 'fp16': False, 'train_batch_size': 8, 'gradient_accumulation_steps': 4, 'evaluate_during_training': True, 'max_seq_length': 64, 'learning_rate': 0.0001, 'early_stopping_consider_epochs': True, 'early_stopping_delta': 0.01, 'early_stopping_metric': 'acc', 'early_stopping_metric_minimize': False, 'early_stopping_patience': 3, 'evaluate_during_training_steps': 331, 'output_dir': 'outputs/ber